In [1]:
#Import the values
from lang_sam import LangSAM
import numpy as np
from PIL import Image, ImageDraw, ImageFont

/home/misango/anaconda3/envs/sam_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/misango/anaconda3/envs/sam_env/lib/python3.11/site-packages/sam2/modeling/sam/transformer.py:23: UserWarning: Flash Attention is disabled as it requires a GPU with Ampere (8.0) CUDA capability.
  OLD_GPU, USE_FLASH_ATTN, MATH_KERNEL_ON = get_sdpa_settings()


In [2]:
model = LangSAM()

In [3]:
# Allow image upload here and save it as a file i.e original_image = st.file_uploader("Drop the image", type=['png', 'jpg', 'jpeg'])
# Load the image
original_image = Image.open('city.jpg')

In [11]:
model = LangSAM()

image_pil = original_image.convert("RGB")
num_objects = input("How many things in the image would you like to label?")
if num_objects:
    num_objects = int(num_objects)  # Convert the input to an integer
    print(f"Number of objects you want to label are confirmed as {num_objects}")

    # Collecting text prompts from the user
    text_prompts = []
    for i in range(num_objects):
        text_prompt = input(f"Enter the thing you want to label (Label {i+1})")
        if text_prompt:
            text_prompts.append(text_prompt)

    if len(text_prompts) == num_objects:
        try:
            # Perform predictions for each text prompt
            predictions = []
            for text_prompt in text_prompts:
                print(f"Predicting for prompt: {text_prompt}")
                results = model.predict([image_pil], [text_prompt])
                print(f"Raw prediction results: {results}")
                for result in results:
                    if "masks" in result and "boxes" in result and "scores" in result:
                        masks = result["masks"]
                        boxes = result["boxes"]
                        phrases = [text_prompt] * len(boxes)  # Using the text prompt as the phrase
                        logits = result["scores"]
                        print(f"Prediction result: masks={masks}, boxes={boxes}, phrases={phrases}, logits={logits}")
                        predictions.append((masks, boxes, phrases, logits))
                    else:
                        print(f"Unexpected prediction result format: {result}")
                        continue  # Skip this prediction if the format is unexpected

            # Draw predictions for each text prompt on the input image
            image_array = np.asarray(image_pil)
            draw = ImageDraw.Draw(image_pil)
            font = ImageFont.load_default()  # You can choose a custom font if needed

            # Define a list of colors for labeling
            colors = ["red", "green", "blue", "yellow", "purple", "orange", "cyan", "magenta"]

            for i, (masks, boxes, phrases, logits) in enumerate(predictions):
                labels = [f"{phrase} {logit:.2f}" for phrase, logit in zip(phrases, logits)]
                label_color = colors[i % len(colors)]  # Change label color based on the prompt

                for label, box in zip(labels, boxes):
                    x0, y0, x1, y1 = box  # Unpack the coordinates
                    draw.rectangle([x0, y0, x1, y1], outline=label_color)  # Draw the rectangle
                    draw.text((x0, y0), label, fill=label_color, font=font)  # Draw the label
        except TypeError as e:
            print(f"TypeError in model.predict: {e}")
        except Exception as e:
            print(f"An error occurred during prediction: {e}")

Number of objects you want to label are confirmed as 1
Predicting for prompt: bird
Predicting 1 masks


/home/misango/anaconda3/envs/sam_env/lib/python3.11/site-packages/torchvision/transforms/functional.py:154: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


Predicted 1 masks
Raw prediction results: [{'scores': array([0.5679366], dtype=float32), 'labels': ['bird'], 'boxes': array([[1013.1212, 1820.2561, 1057.6545, 1881.7363]], dtype=float32), 'masks': array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32), 'mask_scores': array(0.87109375, dtype=float32)}]
Prediction result: masks=[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]], boxes=[[1013.1212 1820.2561 1057.6545 1881.7363]], phrases=['bird'], logits=[0.5679366]


In [13]:
# Display the image with predictions
image_pil.show()
